In [52]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import ta
from ta import add_all_ta_features
from ta.utils import dropna
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from main import add_classic_indicators, get_euronext_tickers, get_stock_data, add_fin_ratios_and_commodities
from alpha101 import add_artificial_variables
from arch import arch_model
import itertools
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [67]:
euro_tickers = get_euronext_tickers()
data = get_stock_data(euro_tickers[0])
data.head()

,Open,High,Low,Close,Volume,Returns,Log Returns
2018-06-22,9.00,9.68,8.90,9.50,677679,NaN,NaN
2018-06-25,9.66,9.80,9.10,9.32,102472,-0.018947,-0.019129
2018-06-26,9.56,9.56,9.14,9.22,35033,-0.010730,-0.010788
2018-06-27,9.20,9.34,9.10,9.28,53496,0.006508,0.006486
2018-06-28,9.28,9.28,9.20,9.28,4160,0.000000,0.000000


,Open,High,Low,Close,Volume,Returns,Log Returns
2017-11-28,2.050,2.073,2.038,2.068,0,NaN,NaN
2017-11-29,2.088,2.116,2.088,2.093,0,0.012089,0.012016
2017-11-30,2.109,2.156,2.104,2.144,0,0.024367,0.024075
2017-12-01,2.124,2.171,2.068,2.118,0,-0.012127,-0.012201
2017-12-04,2.148,2.161,2.139,2.148,0,0.014164,0.014065
...,...,...,...,...,...,...,...
2022-11-21,3.996,4.022,3.945,4.008,0,0.002000,0.001998
2022-11-22,3.957,3.969,3.929,3.938,0,-0.017465,-0.017619
2022-11-23,3.972,4.000,3.889,3.892,0,-0.011681,-0.011750
2022-11-25,3.917,3.951,3.859,3.870,0,-0.005653,-0.005669


In [68]:
add_fin_ratios_and_commodities(data)

,Open,High,Low,Close,Volume,Returns,Log Returns,Gold Close,Oil Close
2018-06-22,9.000,9.680,8.900,9.500,677679,NaN,NaN,1267.400024,68.580002
2018-06-25,9.660,9.800,9.100,9.320,102472,-0.018947,-0.019129,1265.599976,68.080002
2018-06-26,9.560,9.560,9.140,9.220,35033,-0.010730,-0.010788,1256.599976,70.529999
2018-06-27,9.200,9.340,9.100,9.280,53496,0.006508,0.006486,1252.800049,72.760002
2018-06-28,9.280,9.280,9.200,9.280,4160,0.000000,0.000000,1247.800049,73.449997
...,...,...,...,...,...,...,...,...,...
2022-11-21,2.660,2.660,2.560,2.600,5488,-0.001919,-0.001921,1737.400024,79.730003
2022-11-22,2.600,2.625,2.560,2.625,2857,0.009615,0.009569,1738.300049,80.949997
2022-11-23,2.560,2.600,2.560,2.565,4544,-0.022857,-0.023122,1744.900024,77.940002
2022-11-24,2.565,2.635,2.560,2.565,3833,0.000000,0.000000,NaN,NaN


In [64]:
data.join(get_stock_data('GC=F')["Close"])


ValueError: columns overlap but no suffix specified: Index(['Close'], dtype='object')

In [56]:
add_fin_ratios_and_commodities(data).head()

,Open,High,Low,Close,Volume,Returns,Log Returns,Gold Close
Date,,,,,,,,
2018-06-22 00:00:00+02:00,9.00,9.68,8.90,9.50,677679,NaN,NaN,NaN
2018-06-25 00:00:00+02:00,9.66,9.80,9.10,9.32,102472,-0.018947,-0.019129,NaN
2018-06-26 00:00:00+02:00,9.56,9.56,9.14,9.22,35033,-0.010730,-0.010788,NaN
2018-06-27 00:00:00+02:00,9.20,9.34,9.10,9.28,53496,0.006508,0.006486,NaN
2018-06-28 00:00:00+02:00,9.28,9.28,9.20,9.28,4160,0.000000,0.000000,NaN


In [50]:
data = add_classic_indicators(data)
data.tail()

,Open,High,Low,Close,Volume,Returns,Log Returns,adx 4,ema7,ema30,ema50,ema100,ema150,ema200,sma7,sma30,sma50,sma100,sma150,sma200,macd_24_52,macd_12_26,adx5,rsi3,rsi5,rsi7,rsi10,rsi14,rsi17,rsi20,rsi25,rsi30,stochrsi4,stochrsi7,stochrsi10,stochrsi14,stochrsi20,stochrsi25,willr7,willr14,...,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx_pos,trend_adx_neg,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-21 00:00:00+01:00,2.660,2.660,2.560,2.600,5488.0,-0.001919,-0.001921,48.819038,2.599915,2.534516,2.596493,2.873408,3.125877,3.322068,2.646429,2.482833,2.4888,2.81280,3.310733,3.512875,0.032487,0.019290,24.978470,43.712080,53.760992,56.939230,57.442378,55.741702,54.111850,52.553414,50.336740,48.624364,0.261964,0.095668,0.298942,0.607749,0.677744,0.715105,-56.000042,-31.818202,...,0.135545,24.039407,-0.15800,61.909435,21.978817,39.930618,2.570,2.52,2.5450,2.5025,100.0,25.553685,13.751616,2.41125,2.745,80.0,56.0,24.0,2.379741,NaN,0.0,0.0,0.596949,0.682089,11.391290,60.707526,68.181798,67.045445,0.175794,10.169492,1.533310,0.768950,0.764360,-18.397690,-12.559429,-5.838261,2.583440,-0.191943,-0.192127,-72.631580
2022-11-22 00:00:00+01:00,2.600,2.625,2.560,2.625,2857.0,0.009615,0.009569,42.632956,2.606186,2.540354,2.597611,2.868489,3.119242,3.315132,2.637143,2.492833,2.4837,2.80745,3.296767,3.504775,0.031253,0.016542,25.339055,59.946820,60.333671,60.960819,60.036059,57.552475,55.603883,53.825628,51.356975,49.471402,1.000000,0.250463,0.418349,0.681385,0.749166,0.785874,-63.888870,-26.136362,...,0.162264,23.931787,-0.06025,70.903516,31.461507,39.442009,2.570,2.52,2.5450,2.5025,100.0,24.442585,13.153682,2.41125,2.745,76.0,52.0,24.0,2.394151,NaN,0.0,0.0,0.638328,0.632526,12.038325,56.504431,73.863638,70.454541,0.146912,11.702132,1.583820,0.931924,0.651896,-21.800100,-14.407563,-7.392537,2.587214,0.961542,0.956949,-72.368421
2022-11-23 00:00:00+01:00,2.560,2.600,2.560,2.565,4544.0,-0.022857,-0.023122,37.993395,2.595890,2.541944,2.596332,2.862479,3.111901,3.307668,2.617857,2.502333,2.4781,2.80190,3.282800,3.496900,0.027790,0.009907,25.673884,29.409939,42.296818,48.324071,51.643048,52.048560,51.346644,50.410834,48.848256,47.525999,0.000000,0.000000,0.000000,0.457565,0.557461,0.611857,-97.222160,-41.666648,...,0.180002,23.773793,-0.06225,77.937013,41.543889,36.393124,2.615,2.52,2.5675,2.5025,100.0,23.349241,12.565303,2.45625,2.745,72.0,48.0,24.0,2.407985,NaN,0.0,0.0,0.582233,0.605837,11.004978,50.338484,58.333352,66.792929,0.120265,4.693878,1.419053,1.029350,0.389703,-22.445177,-16.015086,-6.430091,2.586172,-2.285712,-2.312240,-72.999999
2022-11-24 00:00:00+01:00,2.565,2.635,2.560,2.565,3833.0,0.000000,0.000000,42.498742,2.588167,2.543431,2.595104,2.856589,3.104658,3.300279,2.598571,2.512833,2.4735,2.79775,3.268100,3.488975,0.024851,0.004991,26.321463,29.409939,42.296818,48.324071,51.643048,52.048560,51.346644,50.410834,48.848256,47.525999,0.000000,0.000000,0.000000,0.210801,0.557461,0.611857,-97.222160,-46.052605,...,0.190600,23.491132,-0.07700,85.312672,51.040728,34.271944,2.650,2.52,2.5850,2.4750,100.0,24.577054,11.903661,2.44375,2.745,68.0,44.0,24.0,2.421266,NaN,0.0,0.0,0.449917,0.556826,10.162399,44.188770,53.947395,62.048128,0.109632,4.693878,1.273836,1.078247,0.195589,-23.770986,-17.566266,-6.204720,2.585588,0.000000,0.000000,-72.999999
2022-11-25 00:00:00+01:00,2.565,2.565,2.565,2.565,0.0,0.000000,0.000000,45.877753,2.582376,2

In [41]:
data = add_artificial_variables(data)
data.tail()

,Open,High,Low,Close,Volume,Returns,Log Returns,adx 4,ema7,ema30,ema50,ema100,ema150,ema200,sma7,sma30,sma50,sma100,sma150,sma200,macd_24_52,macd_12_26,adx5,rsi3,rsi5,rsi7,rsi10,rsi14,rsi17,rsi20,rsi25,rsi30,stochrsi4,stochrsi7,stochrsi10,stochrsi14,stochrsi20,stochrsi25,willr7,willr14,...,alpha33,alpha34,alpha35,alpha36,alpha37,alpha38,alpha40,alpha41,alpha42,alpha43,alpha44,alpha45,alpha47,alpha49,alpha50,alpha51,alpha52,alpha53,alpha54,alpha55,alpha60,alpha61,alpha62,alpha64,alpha65,alpha68,alpha71,alpha74,alpha75,alpha78,alpha81,alpha83,alpha84,alpha85,alpha86,alpha94,alpha95,alpha96,alpha99,alpha101
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-11-21 00:00:00+01:00,2.660,2.660,2.560,0.026391,5488.0,-0.001919,-0.001921,48.819038,2.599915,2.534516,2.596493,2.873408,3.125877,3.322068,2.646429,2.482833,2.4888,2.81280,3.310733,3.512875,0.032487,0.019290,24.978470,43.712080,53.760992,56.939230,57.442378,55.741702,54.111850,52.553414,50.336740,48.624364,0.261964,0.095668,0.298942,0.607749,0.677744,0.715105,-56.000042,-31.818202,...,0.636559,0.786177,676.0,3.401855,0.979391,-0.243599,-0.084550,-0.010479,14.117647,120.0,-0.722237,2.205795e-01,0.033552,2.578609,-0.960428,2.578609,0.126286,15.854965,2.635498e+11,-0.415832,-0.001034,0,-1,0,0,0,0.673426,-1,0,0.458311,-1,0.504918,0.151214,0.483960,0,-0.494585,1,-0.673426,-1,-26.075300
2022-11-22 00:00:00+01:00,2.600,2.625,2.560,2.625000,2857.0,0.009615,0.009569,42.632956,2.606186,2.540354,2.597611,2.868489,3.119242,3.315132,2.637143,2.492833,2.4837,2.80745,3.296767,3.504775,0.031253,0.016542,25.339055,59.946820,60.333671,60.960819,60.036059,57.552475,55.603883,53.825628,51.356975,49.471402,1.000000,0.250463,0.418349,0.681385,0.749166,0.785874,-63.888870,-26.136362,...,0.235484,0.352052,528.0,3.437827,0.765412,-0.394732,-0.094985,-0.010204,0.488864,15.0,-0.643791,7.814986e-03,-0.063941,-2.598609,-0.960428,-2.598609,0.031543,-14.788607,-9.532792e-01,0.036937,-0.002320,0,-1,-1,0,0,0.765591,0,0,0.346348,-1,0.302732,0.253863,0.422042,0,-0.571582,1,-0.765591,-1,0.378789
2022-11-23 00:00:00+01:00,2.560,2.600,2.560,0.026527,4544.0,-0.022857,-0.023122,37.993395,2.595890,2.541944,2.596332,2.862479,3.111901,3.307668,2.617857,2.502333,2.4781,2.80190,3.282800,3.496900,0.027790,0.009907,25.673884,29.409939,42.296818,48.324071,51.643048,52.048560,51.346644,50.410834,48.848256,47.525999,0.000000,0.000000,0.000000,0.457565,0.557461,0.611857,-97.222160,-41.666648,...,0.626882,0.448164,27.0,3.718397,1.018011,-0.135595,-0.042998,0.008672,15.419355,96.0,-0.009386,7.617995e-02,0.069773,2.598473,-0.960428,2.598473,0.109714,10.489925,5.301512e+11,0.793243,-0.001122,0,-1,-1,0,0,0.438634,0,0,0.440430,-1,0.712568,0.162252,0.871872,0,-0.517761,1,-0.438634,-1,-61.792076
2022-11-24 00:00:00+01:00,2.565,2.635,2.560,2.565000,3833.0,0.000000,0.000000,42.498742,2.588167,2.543431,2.595104,2.856589,3.104658,3.300279,2.598571,2.512833,2.4735,2.79775,3.268100,3.488975,0.024851,0.004991,26.321463,29.409939,42.296818,48.324071,51.643048,52.048560,51.346644,50.410834,48.848256,47.525999,0.000000,0.000000,0.000000,0.210801,0.557461,0.611857,-97.222160,-46.052605,...,0.372581,0.160907,1050.0,3.386441,0.855914,-0.227745,-0.016231,0.015979,0.901602,40.0,-0.051845,1.240256e-01,-0.045730,-2.538473,-0.960428,-2.538473,0.030300,-181.008591,-6.666815e-02,0.753115,-0.002218,0,-1,-1,0,0,0.628495,0,0,0.275290,-1,0.896175,0.824503,0.846382,0,-0.233594,1,-0.628495,-1,0.000000
2022-11-25 00:00:00+01:00,2.565,2.565,2.565,2.565000,0.0,0.000000,0.000000,45.877753,2.582376,2.544823,2.593923,2.850815,3.097510,3.292962,2.586429,2.518000,2.4686,2.79085,3.253867,3.480625,0.022340,0.001398,26.922785,29.409939,42.296818,48.324071,51.643048,52.048560,51.346644,50.410834,48.848256,47.525999,0.000000,0.000000,0.000000,0.210801,0.557461,0.611857,-94.999893,-50.000000,...,0.372581,0.942225,116.0,3.626630,0.872581,-0.200375,-0.061469,0.000000,0.777136,2.0,-0.768021,2.002301e-08,-0.000000,-0.000000,-0.935829,-0.0

In [46]:
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas_datareader.data as web
today = dt.datetime.now()
start =  today - relativedelta(years=5)
wti_oil_price = web.DataReader('DCOILWTICO', 'fred', start, today)
wti_oil_price.tail()

,DCOILWTICO
DATE,
2022-11-15,86.87
2022-11-16,85.62
2022-11-17,81.69
2022-11-18,80.07
2022-11-21,79.74


In [ ]:
#determiner
#rolling beta sur plusieurs unités de temps dynamic beta
#https://goldinlocks.github.io/ARCH_GARCH-Volatility-Forecasting/
https://github.com/Taaniya/Stock-Price-Returns-Prediction/blob/master/Prediction_AXISBANK.ipynb

In [ ]:
#https://pub.towardsai.net/statistical-forecasting-for-time-series-data-part-5-arma-garch-model-for-time-series-forecasting-98beeedcfba8
#https://github.com/yashveersinghsohi/Statistical_Modeling_for_Time_Series_Forecasting/tree/master/Returns%20Models
#https://pub.towardsai.net/statistical-forecasting-for-time-series-data-part-6-forecasting-non-stationary-time-series-using-9acc28c39db9
#https://stackoverflow.com/questions/55882111/arima-model-for-certain-lags
#garch sur residuals de l'arima
#tester oubli de certains lags et ajout optimisation des parametres
#https://ionides.github.io/531w18/midterm_project/project38/Midterm_proj.html

In [ ]:
#stationnaire => pas de saisonnalité, aléatoire mais de moyenne constante, ne dépend pas du temps
#test de stationnarité : si la série est non stationnaire => elle possede une tendance et saisonnalité, et dépend du temps (p>0.05)

AdfResult1 = adfuller(data['Close'])
print ("Original P value:",AdfResult1[1]) 

#donc la série est non stationnaire et on doit la différencier pour utiiser arima
data['log_diff_data'] = np.log(data['Close'].values)
data['log_diff_data'] = data['log_diff_data'].diff(1)
AdfResult2 = adfuller(data['log_diff_data'].dropna())
print ("Differenced p value:",AdfResult2[1]) 

plot_acf(data['log_diff_data'].dropna()**2, lags=150,zero=False)
plt.show()

plot_pacf(data['log_diff_data'].dropna()**2, lags=150,zero=False)
plt.show()

In [ ]:
pacf_auto_corr, pacf_conf_int = pacf(data['log_diff_data'].dropna()**2, alpha=.05, nlags=365)
pacf_blue_area = pacf_conf_int - np.array([pacf_auto_corr,pacf_auto_corr]).T
neg_pacf_blue_area, pos_pacf_blue_area = pacf_blue_area.T
pacf_sig_lags = np.where(np.abs(pacf_auto_corr) > 2*np.abs(pos_pacf_blue_area[5]))[0]
pacf_usable_lags_binary = np.zeros(np.max(pacf_sig_lags)+1)
pacf_usable_lags_binary[significant_index] = 1
print(pacf_sig_lags)
plt.plot(neg_pacf_blue_area)
plt.plot(pacf_auto_corr)
plt.plot(pos_pacf_blue_area)
plt.show()

acf_auto_corr, acf_conf_int = acf(data['log_diff_data'].dropna()**2, alpha=.05, nlags=365, fft=True)
acf_blue_area = acf_conf_int - np.array([acf_auto_corr,acf_auto_corr]).T
neg_acf_blue_area, pos_acf_blue_area = acf_blue_area.T
acf_sig_lags = np.where(np.abs(acf_auto_corr) > 2*np.abs(pos_acf_blue_area[5]))[0]
acf_usable_lags_binary = np.zeros(np.max(acf_sig_lags)+1)
acf_usable_lags_binary[significant_index] = 1
print(acf_sig_lags)
plt.plot(neg_acf_blue_area)
plt.plot(acf_auto_corr)
plt.plot(pos_acf_blue_area)
plt.show()

In [ ]:
significant_index

In [ ]:
usable_lags_binary

In [ ]:
stats = pd.concat([model_fit.pvalues, model_fit.tvalues, pd.Series({'bic' : model_fit.bic}), pd.Series({'aic' : model_fit.aic})])

In [ ]:
model_fit.bic, model_fit.aic

In [ ]:
differenciation de la série tant que pas stationnaire

In [ ]:
train_df = data['log_diff_data'].loc[:"2022-05-31"]
test_df = data['log_diff_data'].loc["2022-06-01":]
test_df.shape, train_df.shape

In [ ]:
data['log_diff_data'].mean()

In [ ]:
feature garch n+1

In [ ]:
res = sm.tsa.seasonal_decompose(data['Close'],freq=365)
fig = res.plot()
fig.set_figheight(8)
fig.set_figwidth(15)
plt.show()

In [ ]:
corr_matrix = data.corr()
upperMatrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
corrFeatures = [column for column in upperMatrix.columns if any(upperMatrix[column] > 0.90)]
corrFeatures

In [ ]:
data.isna().sum().max()

In [ ]:
series lagguées a ajouter (come prix j-1 prix j-2)
https://github.com/GoldinLocks/Cryptocurrency-Research-/blob/master/predicting-crypto-prices-with-deep-learning.ipynb
    https://www.srose.biz/wp-content/uploads/2020/08/Deep-Learning-Performance-Part-3-Batch-Normalization-Dropout-Noise.html
    garch s


In [ ]:
calmar

In [ ]:
estimer risk ajuster avec medaf + vol